<a href="https://colab.research.google.com/github/chanukaDissanayaka/RideFareClassification/blob/master/Ridefare_submission_33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import Normalizer

from datetime import datetime, timedelta
from pandas import DataFrame

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Convolution2D,MaxPooling2D,BatchNormalization,Convolution1D,Conv1D,MaxPooling1D
from keras.optimizers import SGD,RMSprop

RANDOM_SEED = 4

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#dataframe = pd.read_csv("/content/train.csv")
#test = pd.read_csv("/content/test.csv")

dataframe = pd.read_csv("/content/gdrive/My Drive/ML_RideFare/Data/train.csv")
test = pd.read_csv("/content/gdrive/My Drive/ML_RideFare/Data/test.csv")

dataframe.isna().sum(axis = 0)
#dataframe = dataframe.fillna(method='ffill')
#dataframe = dataframe.dropna()

tripid                         0
additional_fare              202
duration                     202
meter_waiting                202
meter_waiting_fare           202
meter_waiting_till_pickup    202
pickup_time                    0
drop_time                      0
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         137
label                          0
dtype: int64

In [ ]:
def preprocess_train_data(dataframe):
  feature_set = dataframe.iloc[:,1:13]

  features = feature_set

  features['pickup_time'] = pd.to_datetime(features['pickup_time'], format="%m/%d/%Y %H:%M")
  features['drop_time'] = pd.to_datetime(features['drop_time'], format="%m/%d/%Y %H:%M")

  td = features

  td['pickup_time_year'] = td['pickup_time'].dt.year
  td['pickup_time_month'] = td['pickup_time'].dt.month
  td['pickup_time_day'] = td['pickup_time'].dt.day
  td['pickup_time_hour'] = td['pickup_time'].dt.hour
  td['pickup_time_minute'] = td['pickup_time'].dt.minute

  td['drop_time_year'] = td['drop_time'].dt.year
  td['drop_time_month'] = td['drop_time'].dt.month
  td['drop_time_day'] = td['drop_time'].dt.day
  td['drop_time_hour'] = td['drop_time'].dt.hour
  td['drop_time_minute'] = td['drop_time'].dt.minute

  td['day_of_week'] = td['pickup_time'].dt.day_name()
  td['day_of_week'] = OrdinalEncoder().fit_transform(td[['day_of_week']])

  td['duration_calculated'] = td['drop_time'] - td['pickup_time']
  td['duration_calculated'] = td['duration_calculated'].dt.total_seconds().astype(int)

  td = td.drop('pickup_time',1)
  td = td.drop('drop_time',1)

  td = get_trip_distance_eucld(td)
  td = get_X_distance(td)
  td = get_Y_distance(td)
  td = get_pick_XY(td)
  td = get_drop_XY(td)
  td = transform_hour(td)
  td = set_fare_duration(td)
  td = calculate_direction(td)
  td = set_driving_duration(td)
  td = get_exact_fare(td)
  td = get_manhtn_distance(td)
  td = get_distance_fare_rate(td)
  td = get_time_fare_rate(td)
  

  return td

def get_labels(dataframe):
  label_set = dataframe.iloc[:, 13:14]
  #rev_label_set = label_set
  rev_label_set = label_set.replace({'correct':'incorrect', 'incorrect':'correct'})

  encoder = LabelEncoder()
  encoder.fit(rev_label_set)
  labels = encoder.transform(rev_label_set)
  return labels

def set_time_of_the_day(cleaned_train_set):
  cleaned_train_set.head()
  newFrame = cleaned_train_set
  tt = newFrame

  d0_index = newFrame[(newFrame['pickup_time_hour']>=0) & (newFrame['pickup_time_hour'] < 6 )].index
  tt.loc[d0_index,'time_of_day']=0

  d1_index = newFrame[(newFrame['pickup_time_hour']>=6) & (newFrame['pickup_time_hour'] < 12 )].index
  tt.loc[d1_index,'time_of_day']=1

  d2_index = newFrame[(newFrame['pickup_time_hour']>=12) & (newFrame['pickup_time_hour'] < 18 )].index
  tt.loc[d2_index,'time_of_day']=2

  d3_index = newFrame[(newFrame['pickup_time_hour']>=18) & (newFrame['pickup_time_hour'] < 24 )].index
  tt.loc[d3_index,'time_of_day']=3

  return tt

def haversine_distance(lat1, lon1, lat2, lon2):
  r = 6371
  phi1 = np.radians(lat1)
  phi2 = np.radians(lat2)
  d_phi = np.radians(lat2-lat1)
  d_lambda = np.radians(lon2-lon1)

  a = np.sin(d_phi /2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(d_lambda/2)**2
  res = r*(2* np.arctan2(np.sqrt(a), np.sqrt(1-a)))

  return np.round(res,2)
  return res

def get_trip_distance(frame):
  newFrame = frame
  newFrame["distance"] = haversine_distance(frame['pick_lat'],frame['drop_lat'],frame['pick_lon'],frame['drop_lon'])
  return newFrame

def get_trip_distance_eucld(frame):
  newFrame = frame
  newFrame["distance"] = (((frame['pick_lat']-frame['drop_lat'])**2)+((frame['pick_lon']-frame['drop_lon'])**2)**0.5)
  return newFrame

def get_X_distance(frame):
  newFrame = frame
  newFrame["distance_X"] = ((frame['pick_lat']-frame['drop_lat']))
  return newFrame

def get_Y_distance(frame):
  newFrame = frame
  newFrame["distance_Y"] = ((frame['pick_lon']-frame['drop_lon']))
  return newFrame

def get_manhtn_distance(frame):
  newFrame = frame
  newFrame["distance_mnhtn"] = ((abs(frame['distance_X'])+abs(frame['distance_Y'])))
  return newFrame

def get_pick_XY(frame):
  newFrame = frame
  newFrame["pick_X"] = np.cos(frame['pick_lat'])* np.cos(frame['pick_lon'])
  newFrame["pick_Y"] = np.cos(frame['pick_lat'])* np.sin(frame['pick_lon'])
  return newFrame

def get_drop_XY(frame):
  newFrame = frame
  newFrame["drop_X"] = np.cos(frame['drop_lat'])* np.cos(frame['drop_lon'])
  newFrame["drop_Y"] = np.cos(frame['drop_lat'])* np.sin(frame['drop_lon'])
  return newFrame

def transform_hour(frame):
  newFrame = frame
  newFrame["pick_hour_sin"] = np.sin(np.radians(3*frame['pickup_time_hour']))
  newFrame["drop_hour_sin"] = np.sin(np.radians(3*frame['drop_time_hour']))

  newFrame["pick_hour_cos"] = np.cos(np.radians(3*frame['pickup_time_hour']))
  newFrame["drop_hour_cos"] = np.cos(np.radians(3*frame['drop_time_hour']))
  return newFrame

def set_fare_duration(imputed_train_set):
  df = imputed_train_set

  df['total_fare'] = ((df['fare']-df['fare'].mean())/df['fare'].std() +
                      (df['additional_fare']-df['additional_fare'].mean())/df['additional_fare'].std()+ 
                      (df['meter_waiting_fare']-df['meter_waiting_fare'].mean())/df['meter_waiting_fare'].std())

  df['total_fare'] =  df['fare'] + df['additional_fare'] + df['meter_waiting_fare']
  df['total_additional_fare'] = df['additional_fare'] + df['meter_waiting_fare']
  df['total_waiting_time'] = df['meter_waiting'] + df['meter_waiting_till_pickup']
  df['total_time'] = df['total_waiting_time'] + df['duration']

  df['waiting_factor'] = ((df['meter_waiting']-df['meter_waiting'].mean())/df['meter_waiting'].std() +
                      (df['meter_waiting_fare']-df['meter_waiting_fare'].mean())/df['meter_waiting_fare'].std()+ 
                      (df['meter_waiting_till_pickup']-df['meter_waiting_till_pickup'].mean())/df['meter_waiting_till_pickup'].std())

  return df

def calculate_direction(frame):
    newFrame = frame

    distance_X = frame['distance_X']
    distance_Y = frame['distance_Y']

    result = np.zeros(len(distance_X))
    l = np.sqrt(distance_X**2 + distance_Y**2)
    result[distance_X>0] = (180/np.pi)*np.arcsin(distance_Y[distance_X>0]/l[distance_X>0])
    idx = (distance_X<0) & (distance_Y>0)
    result[idx] = 180 - (180/np.pi)*np.arcsin(distance_Y[idx]/l[idx])
    idx = (distance_X<0) & (distance_Y<0)
    result[idx] = -180 - (180/np.pi)*np.arcsin(distance_Y[idx]/l[idx])
    newFrame['direction'] = result
    return newFrame

def set_driving_duration(train_set):
  df = train_set

  df['driving_duration'] = df['duration'] - df['meter_waiting']
  return df

def get_exact_fare(train_set):
  df = train_set
  df['exact_fare'] = df['fare'] - df['additional_fare']- df['meter_waiting_fare']
  return df

def get_time_fare_rate(train_set):
  df = train_set
  df['time_fare_rate'] = df['driving_duration']/ (df['exact_fare']+1)
  return df

def get_distance_fare_rate(train_set):
  df = train_set
  df['distance_fare_rate'] = df['distance']/ (df['exact_fare']+1)
  return df

In [ ]:
print(dataframe.label.value_counts())

correct      15495
incorrect     1681
Name: label, dtype: int64


In [ ]:
cleaned_train_set = preprocess_train_data(dataframe)
labels = get_labels(dataframe)
cleaned_train_set.head()


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_year,pickup_time_month,pickup_time_day,pickup_time_hour,pickup_time_minute,drop_time_year,drop_time_month,drop_time_day,drop_time_hour,drop_time_minute,day_of_week,duration_calculated,distance,distance_X,distance_Y,pick_X,pick_Y,drop_X,drop_Y,pick_hour_sin,drop_hour_sin,pick_hour_cos,drop_hour_cos,total_fare,total_additional_fare,total_waiting_time,total_time,waiting_factor,direction,driving_duration,exact_fare,distance_mnhtn,distance_fare_rate,time_fare_rate
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,2019,11,1,0,20,2019,11,1,0,34,0.0,840,0.022663,-0.04078,0.0210,-0.175519,-0.818213,-0.187363,-0.791950,0.000000,0.000000,1.000000,1.000000,280.8200,10.5000,120.0,954.0,-0.249582,152.753420,778.0,259.8200,0.06178,0.000087,2.982900
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,2019,11,1,0,56,2019,11,1,1,9,0.0,780,0.006875,-0.02784,0.0061,-0.173513,-0.805325,-0.174935,-0.788535,0.000000,0.052336,1.000000,0.998630,208.3500,10.5000,181.0,972.0,-0.092347,167.641274,744.0,187.3500,0.03394,0.000037,3.950093
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,2019,11,1,1,8,2019,11,1,1,26,0.0,1080,0.050301,-0.02830,-0.0495,-0.197078,-0.786528,-0.154631,-0.778754,0.052336,0.052336,0.998630,0.998630,312.1400,10.5000,141.0,1228.0,-0.253392,-119.757345,1007.0,291.1400,0.07780,0.000172,3.446977
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,2019,11,1,2,27,2019,11,1,2,37,0.0,600,0.007603,-0.00178,-0.0076,-0.175582,-0.781100,-0.169414,-0.781369,0.104528,0.104528,0.994522,0.994522,108.4638,26.1638,339.0,937.0,-0.180218,-103.181677,327.0,56.1362,0.00938,0.000133,5.723167
4,NaN,NaN,NaN,NaN,NaN,6.87441,79.8615,6.84478,79.9290,358.39,2019,11,1,3,34,2019,11,1,3,51,0.0,1020,0.068378,0.02963,-0.0675,-0.204695,-0.804630,-0.152875,-0.832487,0.156434,0.156434,0.987688,0.987688,NaN,NaN,NaN,NaN,NaN,-66.300305,NaN,NaN,0.09713,NaN,NaN


In [ ]:
#drop outliers

data_exp = cleaned_train_set
data_exp['labels'] = labels

#data_exp = data_exp[data_exp.duration<=10000]
data_exp = data_exp[data_exp.drop_lon>70.0]
data_exp = data_exp[data_exp.drop_lat<10.0]
data_exp = data_exp.drop(data_exp[(data_exp.fare == 0) & (data_exp.labels == 1)].index)
#data_exp = data_exp[data_exp.meter_waiting<=20000]
#data_exp = data_exp[data_exp.meter_waiting_till_pickup<=1000]
data_exp.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_year,pickup_time_month,pickup_time_day,pickup_time_hour,pickup_time_minute,drop_time_year,drop_time_month,drop_time_day,drop_time_hour,drop_time_minute,day_of_week,duration_calculated,distance,distance_X,distance_Y,pick_X,pick_Y,drop_X,drop_Y,pick_hour_sin,drop_hour_sin,pick_hour_cos,drop_hour_cos,total_fare,total_additional_fare,total_waiting_time,total_time,waiting_factor,direction,driving_duration,exact_fare,distance_mnhtn,distance_fare_rate,time_fare_rate,labels
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,2019,11,1,0,20,2019,11,1,0,34,0.0,840,0.022663,-0.04078,0.0210,-0.175519,-0.818213,-0.187363,-0.791950,0.000000,0.000000,1.000000,1.000000,280.8200,10.5000,120.0,954.0,-0.249582,152.753420,778.0,259.8200,0.06178,0.000087,2.982900,1
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,2019,11,1,0,56,2019,11,1,1,9,0.0,780,0.006875,-0.02784,0.0061,-0.173513,-0.805325,-0.174935,-0.788535,0.000000,0.052336,1.000000,0.998630,208.3500,10.5000,181.0,972.0,-0.092347,167.641274,744.0,187.3500,0.03394,0.000037,3.950093,1
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,2019,11,1,1,8,2019,11,1,1,26,0.0,1080,0.050301,-0.02830,-0.0495,-0.197078,-0.786528,-0.154631,-0.778754,0.052336,0.052336,0.998630,0.998630,312.1400,10.5000,141.0,1228.0,-0.253392,-119.757345,1007.0,291.1400,0.07780,0.000172,3.446977,1
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,2019,11,1,2,27,2019,11,1,2,37,0.0,600,0.007603,-0.00178,-0.0076,-0.175582,-0.781100,-0.169414,-0.781369,0.104528,0.104528,0.994522,0.994522,108.4638,26.1638,339.0,937.0,-0.180218,-103.181677,327.0,56.1362,0.00938,0.000133,5.723167,1
4,NaN,NaN,NaN,NaN,NaN,6.87441,79.8615,6.84478,79.9290,358.39,2019,11,1,3,34,2019,11,1,3,51,0.0,1020,0.068378,0.02963,-0.0675,-0.204695,-0.804630,-0.152875,-0.832487,0.156434,0.156434,0.987688,0.987688,NaN,NaN,NaN,NaN,NaN,-66.300305,NaN,NaN,0.09713,NaN,NaN,1


In [ ]:
print((data_exp.meter_waiting_fare/data_exp.meter_waiting).mean())


0.029266443352009574


In [ ]:
from sklearn.impute import SimpleImputer

data_exp.duration.fillna(data_exp.duration_calculated, inplace=True)
data_exp.additional_fare.fillna(10.5, inplace=True)
data_exp.meter_waiting.fillna(data_exp.meter_waiting.mean(), inplace=True)
data_exp.meter_waiting_fare.fillna(data_exp.meter_waiting*0.03, inplace=True)
data_exp.head()

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
dframe = imp_mean.fit_transform(data_exp)
k = data_exp.keys()
imputed_set = data_exp
imputed_set[k] = dframe
imputed_set.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_year,pickup_time_month,pickup_time_day,pickup_time_hour,pickup_time_minute,drop_time_year,drop_time_month,drop_time_day,drop_time_hour,drop_time_minute,day_of_week,duration_calculated,distance,distance_X,distance_Y,pick_X,pick_Y,drop_X,drop_Y,pick_hour_sin,drop_hour_sin,pick_hour_cos,drop_hour_cos,total_fare,total_additional_fare,total_waiting_time,total_time,waiting_factor,direction,driving_duration,exact_fare,distance_mnhtn,distance_fare_rate,time_fare_rate,labels
0,10.5,834.0,56.000000,0.000000,64.0000,6.86252,79.8993,6.90330,79.8783,270.32,2019.0,11.0,1.0,0.0,20.0,2019.0,11.0,1.0,0.0,34.0,0.0,840.0,0.022663,-0.04078,0.0210,-0.175519,-0.818213,-0.187363,-0.791950,0.000000,0.000000,1.000000,1.000000,280.820000,10.50000,120.000000,954.000000,-0.249582,152.753420,778.000000,259.82000,0.06178,0.000087,2.982900,1.0
1,10.5,791.0,47.000000,0.000000,134.0000,6.88589,79.8984,6.91373,79.8923,197.85,2019.0,11.0,1.0,0.0,56.0,2019.0,11.0,1.0,1.0,9.0,0.0,780.0,0.006875,-0.02784,0.0061,-0.173513,-0.805325,-0.174935,-0.788535,0.000000,0.052336,1.000000,0.998630,208.350000,10.50000,181.000000,972.000000,-0.092347,167.641274,744.000000,187.35000,0.03394,0.000037,3.950093,1.0
2,10.5,1087.0,80.000000,0.000000,61.0000,6.90839,79.8651,6.93669,79.9146,301.64,2019.0,11.0,1.0,1.0,8.0,2019.0,11.0,1.0,1.0,26.0,0.0,1080.0,0.050301,-0.02830,-0.0495,-0.197078,-0.786528,-0.154631,-0.778754,0.052336,0.052336,0.998630,0.998630,312.140000,10.50000,141.000000,1228.000000,-0.253392,-119.757345,1007.000000,291.14000,0.07780,0.000172,3.446977,1.0
3,10.5,598.0,271.000000,15.663800,68.0000,6.92570,79.8895,6.92748,79.8971,82.30,2019.0,11.0,1.0,2.0,27.0,2019.0,11.0,1.0,2.0,37.0,0.0,600.0,0.007603,-0.00178,-0.0076,-0.175582,-0.781100,-0.169414,-0.781369,0.104528,0.104528,0.994522,0.994522,108.463800,26.16380,339.000000,937.000000,-0.180218,-103.181677,327.000000,56.13620,0.00938,0.000133,5.723167,1.0
4,10.5,1020.0,629.095923,18.872878,112.4607,6.87441,79.8615,6.84478,79.9290,358.39,2019.0,11.0,1.0,3.0,34.0,2019.0,11.0,1.0,3.0,51.0,0.0,1020.0,0.068378,0.02963,-0.0675,-0.204695,-0.804630,-0.152875,-0.832487,0.156434,0.156434,0.987688,0.987688,358.462167,45.78192,741.556623,2444.452805,-0.000003,-66.300305,1073.800259,266.91984,0.09713,0.000131,5.209346,1.0


In [ ]:
imputed_set.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_year,pickup_time_month,pickup_time_day,pickup_time_hour,pickup_time_minute,drop_time_year,drop_time_month,drop_time_day,drop_time_hour,drop_time_minute,day_of_week,duration_calculated,distance,distance_X,distance_Y,pick_X,pick_Y,drop_X,drop_Y,pick_hour_sin,drop_hour_sin,pick_hour_cos,drop_hour_cos,total_fare,total_additional_fare,total_waiting_time,total_time,waiting_factor,direction,driving_duration,exact_fare,distance_mnhtn,distance_fare_rate,time_fare_rate,labels
0,10.5,834.0,56.000000,0.000000,64.0000,6.86252,79.8993,6.90330,79.8783,270.32,2019.0,11.0,1.0,0.0,20.0,2019.0,11.0,1.0,0.0,34.0,0.0,840.0,0.022663,-0.04078,0.0210,-0.175519,-0.818213,-0.187363,-0.791950,0.000000,0.000000,1.000000,1.000000,280.820000,10.50000,120.000000,954.000000,-0.249582,152.753420,778.000000,259.82000,0.06178,0.000087,2.982900,1.0
1,10.5,791.0,47.000000,0.000000,134.0000,6.88589,79.8984,6.91373,79.8923,197.85,2019.0,11.0,1.0,0.0,56.0,2019.0,11.0,1.0,1.0,9.0,0.0,780.0,0.006875,-0.02784,0.0061,-0.173513,-0.805325,-0.174935,-0.788535,0.000000,0.052336,1.000000,0.998630,208.350000,10.50000,181.000000,972.000000,-0.092347,167.641274,744.000000,187.35000,0.03394,0.000037,3.950093,1.0
2,10.5,1087.0,80.000000,0.000000,61.0000,6.90839,79.8651,6.93669,79.9146,301.64,2019.0,11.0,1.0,1.0,8.0,2019.0,11.0,1.0,1.0,26.0,0.0,1080.0,0.050301,-0.02830,-0.0495,-0.197078,-0.786528,-0.154631,-0.778754,0.052336,0.052336,0.998630,0.998630,312.140000,10.50000,141.000000,1228.000000,-0.253392,-119.757345,1007.000000,291.14000,0.07780,0.000172,3.446977,1.0
3,10.5,598.0,271.000000,15.663800,68.0000,6.92570,79.8895,6.92748,79.8971,82.30,2019.0,11.0,1.0,2.0,27.0,2019.0,11.0,1.0,2.0,37.0,0.0,600.0,0.007603,-0.00178,-0.0076,-0.175582,-0.781100,-0.169414,-0.781369,0.104528,0.104528,0.994522,0.994522,108.463800,26.16380,339.000000,937.000000,-0.180218,-103.181677,327.000000,56.13620,0.00938,0.000133,5.723167,1.0
4,10.5,1020.0,629.095923,18.872878,112.4607,6.87441,79.8615,6.84478,79.9290,358.39,2019.0,11.0,1.0,3.0,34.0,2019.0,11.0,1.0,3.0,51.0,0.0,1020.0,0.068378,0.02963,-0.0675,-0.204695,-0.804630,-0.152875,-0.832487,0.156434,0.156434,0.987688,0.987688,358.462167,45.78192,741.556623,2444.452805,-0.000003,-66.300305,1073.800259,266.91984,0.09713,0.000131,5.209346,1.0


In [ ]:
numeric_cols = ['meter_waiting','meter_waiting_fare','meter_waiting_till_pickup',
               'additional_fare','duration','pick_lat','pick_lon','drop_lat','drop_lon',
               'fare','pickup_time_day','pickup_time_hour','pickup_time_minute']

numeric_cols = ['meter_waiting','total_additional_fare','meter_waiting_till_pickup',
                'driving_duration','fare','distance','time_fare_rate','distance_fare_rate',
                'distance_X','distance_Y','direction',
                'pickup_time_hour']
categorical_cols = []



In [ ]:
imputed_train_set = imputed_set

imputed_train_labels = imputed_set.labels
imputed_train_set = imputed_set.drop('labels', 1)

In [ ]:
X_train, X_eval, Y_train, Y_eval = train_test_split(
    imputed_train_set,
    imputed_train_labels,
    test_size=0.20,
    shuffle=True,
    stratify=imputed_train_labels,
    random_state=RANDOM_SEED
)



In [ ]:
numeric_preprocessing_steps = Pipeline([
    ('minmax_scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OrdinalEncoder())
    ])


preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols),
        ("categorical", categorical_transformer, categorical_cols)
    ],
    remainder = "drop"
)

data_pipeline = Pipeline([
    ("preprocessor", preprocessor)
])


In [ ]:
data_pipeline.fit(X_train)
d_train = data_pipeline.transform(X_train)
d_eval = data_pipeline.transform(X_eval)

In [ ]:
print(d_train.shape, d_eval.shape)
Y_train.shape

(13739, 12) (3435, 12)


(13739,)

In [ ]:
d_train[0]

array([8.81737022e-05, 4.00624189e-04, 1.15036656e-03, 3.75706919e-02,
       3.61166912e-03, 1.55362378e-02, 1.37362606e-02, 2.75500455e-03,
       5.61131661e-01, 5.75283857e-01, 7.24614580e-01, 5.21739130e-01])

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight
sample_w = compute_sample_weight(class_weight={0:5, 1:0.5}, y=Y_train)
#sample_w = compute_sample_weight(class_weight='balanced', y=Y_train)

In [ ]:
sample_w

array([0.5, 0.5, 0.5, ..., 5. , 0.5, 0.5])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score

gbclf = GradientBoostingClassifier(random_state=4, n_estimators = 5000, max_depth =5, 
                                   subsample = 1.0, verbose = 1,max_features=5 )
gbclf.fit(d_train, Y_train, sample_w)
print(gbclf.score(d_eval, Y_eval))
pr = gbclf.predict(d_eval)
print(f1_score(Y_eval, pr, average='macro'))
confusion_matrix(Y_eval, pr)

      Iter       Train Loss   Remaining Time 
         1           1.2761            3.39m
         2           1.1953            3.28m
         3           1.1266            3.32m
         4           1.0663            3.23m
         5           1.0087            3.21m
         6           0.9577            3.19m
         7           0.9147            3.15m
         8           0.8781            3.15m
         9           0.8458            3.16m
        10           0.8181            3.16m
        20           0.6466            3.09m
        30           0.5747            3.08m
        40           0.5221            3.07m
        50           0.4878            3.06m
        60           0.4560            3.06m
        70           0.4281            3.05m
        80           0.4028            3.04m
        90           0.3780            3.04m
       100           0.3534            3.03m
       200           0.2152            2.95m
       300           0.1437            2.88m
       40

array([[ 229,  107],
       [  52, 3047]])

In [ ]:
0.9537117903930131
0.8584368320062123

array([[ 229,  107],
       [  52, 3047]])

0.9543311726576311
0.8448644735309426

array([[ 194,  111],
       [  44, 3045]])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score

gbclf = GradientBoostingClassifier(random_state=2, n_estimators = 5000, max_depth =5, 
                                   subsample = 1.0, verbose = 1)
gbclf.fit(d_train, Y_train, sample_w)
print(gbclf.score(d_eval, Y_eval))
pr = gbclf.predict(d_eval)
print(f1_score(Y_eval, pr, average='macro'))
confusion_matrix(Y_eval, pr)

      Iter       Train Loss   Remaining Time 
         1           1.2933            6.52m
         2           1.2171            6.07m
         3           1.1530            6.01m
         4           1.0998            6.06m
         5           1.0539            5.98m
         6           1.0122            5.93m
         7           0.9765            5.88m
         8           0.9434            5.85m
         9           0.9159            5.84m
        10           0.8915            5.82m
        20           0.7384            5.78m
        30           0.6391            5.73m
        40           0.5815            5.70m
        50           0.5365            5.68m
        60           0.5005            5.66m
        70           0.4696            5.65m
        80           0.4460            5.64m
        90           0.4242            5.63m
       100           0.3987            5.62m
       200           0.2490            5.48m
       300           0.1704            5.38m
       40

array([[ 219,  117],
       [  34, 3065]])

In [ ]:
0.9513828238719069
0.8373952513273669

array([[ 196,  140],
       [  27, 3072]])

In [ ]:

confusion_matrix(Y_eval, pr)

array([[ 191,  145],
       [  21, 3078]])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbclf = GradientBoostingClassifier(random_state=0, n_estimators = 850, max_depth =8, subsample = 1.0, loss ='exponential')
#gbclf.fit(d_train, Y_train, sample_w)
gbclf.fit(d_train, Y_train,sample_w)
gbclf.score(d_eval, Y_eval)

0.9560989982321745

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate

data_pipeline.fit(imputed_train_set)
d_all = data_pipeline.transform(imputed_train_set)
y_all = imputed_train_labels

sample_w = compute_sample_weight(class_weight={0:5, 1:0.5}, y=y_all)

gbclf = GradientBoostingClassifier(random_state=2, n_estimators = 5000, max_depth =5, 
                                   subsample = 1.0, verbose = 1)

scoring = ['f1_macro', 'f1_micro']

scores = cross_validate(gbclf, d_all, y_all, cv=5, scoring = scoring)
print("f1_micro: %0.4f (+/- %0.4f)" % (scores['test_f1_micro'].mean(), scores['test_f1_micro'].std() * 2))
print("f1_macro: %0.4f (+/- %0.4f)" % (scores['test_f1_macro'].mean(), scores['test_f1_macro'].std() * 2))

      Iter       Train Loss   Remaining Time 
         1           0.5598            6.52m
         2           0.5231            6.47m
         3           0.4977            6.50m
         4           0.4778            6.48m
         5           0.4609            6.46m
         6           0.4478            6.46m
         7           0.4364            6.44m
         8           0.4258            6.41m
         9           0.4161            6.42m
        10           0.4083            6.45m
        20           0.3363            6.38m
        30           0.3025            6.40m
        40           0.2800            6.39m
        50           0.2628            6.37m
        60           0.2418            6.34m
        70           0.2308            6.33m
        80           0.2218            6.31m
        90           0.2104            6.30m
       100           0.1988            6.28m
       200           0.1259            6.13m
       300           0.0826            5.99m
       40

In [ ]:
5, 5000
f1_micro: 0.9510 (+/- 0.0111)
f1_macro: 0.8452 (+/- 0.0265)

In [ ]:
print("f1_micro: %0.4f (+/- %0.4f)" % (scores['test_f1_micro'].mean(), scores['test_f1_micro'].std() * 2))
print("f1_macro: %0.4f (+/- %0.4f)" % (scores['test_f1_macro'].mean(), scores['test_f1_macro'].std() * 2))

f1_micro: 0.9502 (+/- 0.0121)
f1_macro: 0.8407 (+/- 0.0359)


In [ ]:
data_pipeline.fit(imputed_train_set)
d_all = data_pipeline.transform(imputed_train_set)
y_all = imputed_train_labels

sample_w = compute_sample_weight(class_weight={0:5, 1:0.5}, y=y_all)

gbclf = GradientBoostingClassifier(random_state=4, n_estimators = 5000, max_depth =5, 
                                   subsample = 1.0, verbose = 1,max_features=5 )

print(d_all.shape, y_all.shape)

gbclf.fit(d_all, y_all,sample_w)

(17174, 12) (17174,)
      Iter       Train Loss   Remaining Time 
         1           1.2815            3.85m
         2           1.2017            3.78m
         3           1.1342            3.79m
         4           1.0715            3.74m
         5           1.0162            3.81m
         6           0.9684            3.78m
         7           0.9268            3.74m
         8           0.8889            3.76m
         9           0.8568            3.76m
        10           0.8258            3.84m
        20           0.6641            3.81m
        30           0.5886            3.79m
        40           0.5402            3.79m
        50           0.5084            3.79m
        60           0.4784            3.80m
        70           0.4531            3.79m
        80           0.4279            3.79m
        90           0.4067            3.78m
       100           0.3869            3.78m
       200           0.2532            3.68m
       300           0.1774      

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=5, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=5000,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=4, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=1,
                           warm_start=False)

In [ ]:
cleaned_test_data = preprocess_train_data(test)

test_set = data_pipeline.transform(cleaned_test_data)

In [ ]:
test_set.shape

(8576, 12)

In [ ]:
preds = gbclf.predict(test_set)
print(preds)

[1. 1. 1. ... 1. 1. 1.]


In [ ]:
preds.shape

(8576,)

In [ ]:
output = pd.DataFrame(test,columns=['tripid'])
output['prediction'] = preds
output = output.astype({"prediction": int})
output.head()

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1


In [ ]:
output.to_csv('/content/Ridefare_submission_33.csv',index=False)